### Plot data from the netCDF conversions of the original log files
To run this Notebook:
- Install Python 3.8 (e.g from [Anacaonda](https://www.anaconda.com/distribution/))
- `git clone git@bitbucket.org:mbari/auv-python.git`
- `cd auv-python`
- `conda create --name auv-python python=3.8`
- `conda activate auv-python`
- `pip install -r requirements.txt`
- `jupyter notebook`
- Select this file and execute the cells with 'shift-return'

In [ ]:
import os
import xarray as xr
import hvplot.pandas
import ipywidgets as widgets
from logs2netcdfs import BASE_PATH
from logs2netcdfs import LOG_FILES

# Pick auv_name available from locally processed files
auv_picker = widgets.Dropdown(options=[f for f in sorted(os.listdir(BASE_PATH))
                                               if f != '.DS_Store'])
display(auv_picker)

In [ ]:
# Pick mission available for the auv_name
auv_name = auv_picker.value
missions_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs')
mission_picker = widgets.Dropdown(options=sorted(os.listdir(missions_dir)))
display(mission_picker)

In [ ]:
# Read all the netCDF file data into in-memory xarray objects saved in a dictionary
netcdfs_dir = os.path.join(BASE_PATH, auv_name, 'missionnetcdfs', mission_picker.value)
ncs = {}
for lf in LOG_FILES:
    nc_file = lf.replace('.log', '.nc')
    try:
        ncs[nc_file] = xr.open_dataset(os.path.join(netcdfs_dir, nc_file))
    except FileNotFoundError:
        pass

In [ ]:
# Show all the attributes of each data set and all the variables in each data set
for nc_file, ds in ncs.items():
    print(f"{nc_file}: {[v for v in list(ds)]}")
    ##display(ds) # Uncomment to explore all the attributes

In [ ]:
# Pick the converted log to examine
ds_picker = widgets.Dropdown(options=ncs.keys())
display(ds_picker)

In [ ]:
# Pick the variable to plot
var_picker = widgets.Dropdown(options=list(ncs[ds_picker.value]))
display(var_picker)

In [ ]:
# Make an interactive plot
ds = ncs[ds_picker.value]
variable = var_picker.value
    
var = getattr(ds, variable)
yl = f"{var_picker.value} from {var.attrs['long_name']} [{var.attrs.get('units', '')}]"
print(f"{auv_name}: {mission_picker.value}: {ds_picker.value}: {var_picker.value}")
display(var.to_pandas().hvplot(ylabel=yl))